In [20]:
import pandas as pd
import tkinter as tk
from tkinter import ttk

# Define global variables
beam_data = None  # Variable to store beam data
depth = None  # Variable to store beam depth
width = None  # Variable to store beam width

# Function to load beam data from a CSV file
def load_beam_data(file_path):
    global beam_data  # Access global variable
    beam_data = pd.read_csv(file_path)

# Function to handle file selection
def select_file():
    if selected_file.get() == 1:
        load_beam_data('structural-sections_SHS.csv')
    elif selected_file.get() == 2:
        load_beam_data('structural-sections_UB.csv')

# Function to get user input
def user_input():
    # Retrieve data from input fields
    applied_force_vertical = float(entry_force_vertical.get())
    applied_force_horizontal = float(entry_force_horizontal.get())
    length = float(entry_length.get())
    global depth, width  # Access global variables
    depth = float(entry_depth.get())
    width = float(entry_width.get())
    deflection_limit_value = entry_deflection_limit.get()
    gamma_M = float(entry_gamma_M.get())
    steel_class = float(entry_steel_class.get())

    if deflection_limit_value.upper() == 'Y':
        deflection_limit = float(length * 1000) / 360
    else:
        deflection_limit = float(deflection_limit_value)

    return applied_force_vertical, applied_force_horizontal, length, deflection_limit, gamma_M, steel_class


def moment_calc(applied_force_vertical, applied_force_horizontal, length):
    # Calculate moments
    moment_y_y = (1/4) * applied_force_vertical * length
    moment_z_z = (1/4) * applied_force_horizontal * length
    return moment_y_y, moment_z_z

def shear_force_calc(applied_force_vertical):
    # Calculate shear force
    shear_force_z_z = applied_force_vertical / 2
    return shear_force_z_z

def moment_resistance_calc(steel_class, gamma_M):
    # Calculate moment resistance based on steel class and safety factor
    beam_data['Moment Resistance y-y'] = (beam_data['Plastic Modulus y-y'] * steel_class) / gamma_M / 1000
    beam_data['Moment Resistance z-z'] = (beam_data['Plastic Modulus z-z'] * steel_class) / gamma_M / 1000

def shear_area_calc():
    # Calculate shear area of the beam
    beam_data['Shear_Area'] = beam_data['Thickness Web'] * beam_data['Depth of section'] - 2 * beam_data['Thickness Flange']

def shear_resistance_calc(steel_class, gamma_M):
    # Calculate shear resistance based on steel class and safety factor
    beam_data['Shear Resistance z-z'] = (beam_data['Shear_Area'] * steel_class) / (3**(1/2) * gamma_M)
    

def calculate_deflection(applied_force, length, elastic_modulus, moment_of_inertia):
    # Calculate beam deflection
    deflection = (applied_force * length**3) * 1000 / (48 * elastic_modulus * moment_of_inertia)
    return deflection

def calculate_deflections(applied_force_vertical, applied_force_horizontal, length):
    # Calculate beam deflections for both axes
    beam_data['Deflection y-y'] = calculate_deflection(applied_force_vertical, length * 1000, 210000,
                                                       beam_data['Second Moment of Area y-y'] * 10000)
    beam_data['Deflection z-z'] = calculate_deflection(applied_force_horizontal, length * 1000, 210000,
                                                       beam_data['Second Moment of Area z-z'] * 10000)

def select_top_three_beams(moment_y_y, moment_z_z, deflection_limit, shear_force_z_z):
    # Select top three beams based on specified criteria
    beam_data['Difference y-y'] = abs(beam_data['Moment Resistance y-y'] - moment_y_y)
    beam_data['Difference z-z'] = abs(beam_data['Moment Resistance z-z'] - moment_z_z)

    beam_data_filtered = beam_data[(beam_data['Deflection y-y'] < deflection_limit) &
                                   (beam_data['Deflection z-z'] < deflection_limit) &
                                   (beam_data['Shear Resistance z-z'] > shear_force_z_z)]
    
    beam_data_filtered = beam_data_filtered[(beam_data_filtered['Depth of section'] < depth) &
                                            (beam_data_filtered['Width of section'] < width)]
    
    top_three_beams = beam_data_filtered.nsmallest(3, ['Difference y-y', 'Difference z-z'])
    return top_three_beams[['Beam_Name', 'Moment Resistance y-y', 'Moment Resistance z-z', 'Difference y-y',
                            'Difference z-z', 'Deflection y-y', 'Deflection z-z']]

def display_calculated_data(top_three_beams, moment_y_y, moment_z_z, shear_force_z_z):
    # Display calculated data including top three beams
    result_text.set("\nThree recommended beams:\n" +
                    str(top_three_beams[['Beam_Name', 'Moment Resistance y-y', 'Deflection y-y',
                                          'Moment Resistance z-z', 'Deflection z-z']]) +
                    f"\n\nCalculated Moment (y-y): {moment_y_y}\nCalculated Moment (z-z): {moment_z_z}"
                    f"\nCalculated Shear Force (z-z): {shear_force_z_z}")

def calculate_beam():
    # Select the file based on user input
    select_file()

    # Get user input from entry widgets
    applied_force_vertical, applied_force_horizontal, length, deflection_limit, gamma_M, steel_class = user_input()

    # Calculate the moment and shear forces and capture the values
    moment_y_y, moment_z_z = moment_calc(applied_force_vertical, applied_force_horizontal, length)
    shear_force_z_z = shear_force_calc(applied_force_vertical)

    # Calculate moment resistance and update beam_data
    moment_resistance_calc(steel_class, gamma_M)

    # Calculate shear resistance and update beam_data
    shear_area_calc()
    shear_resistance_calc(steel_class, gamma_M)

    # Calculate deflections and update beam_data
    calculate_deflections(applied_force_vertical, applied_force_horizontal, length)

    # Select top three beams based on criteria and display in result_label
    top_three_beams = select_top_three_beams(moment_y_y, moment_z_z, deflection_limit, shear_force_z_z)
    display_calculated_data(top_three_beams, moment_y_y, moment_z_z, shear_force_z_z)

# GUI setup
app = tk.Tk()
app.title("Beam Calculator - 'nsmallest'")
window_width = 600
window_height = 2000

# Colors
bg_color = "SkyBlue2"
label_bg_color = "mint cream"
entry_bg_color = "lightcyan"
button_bg_color = "lightgreen"  # Button background color
button_fg_color = "blue4"  # Button foreground color

app.configure(bg=bg_color)

# Font settings
font_size = 12
font_style = ('Arial', font_size = 12)

# Labels for input fields
labels = ["Applied Vertical Force (kN):", "Applied Horizontal Force (kN):", "Length (m):",
          "Max Depth (mm):", "Max Width (mm):", "Deflection Limit (Y for EC limit or other value):", "Partial Safety Factor (gamma_M):",
          "Steel Class (e.g., 235 for S235 steel):"]

# Create labels and entry fields for input
for i, label_text in enumerate(labels):
    ttk.Label(app, text=label_text, background=label_bg_color, font=font_style).grid(row=i, column=0, sticky="e", padx=10, pady=5)

entry_force_vertical = ttk.Entry(app, background=entry_bg_color, font=font_style)
entry_force_horizontal = ttk.Entry(app, background=entry_bg_color, font=font_style)
entry_length = ttk.Entry(app, background=entry_bg_color, font=font_style)
entry_depth = ttk.Entry(app, background=entry_bg_color, font=font_style)
entry_width = ttk.Entry(app, background=entry_bg_color, font=font_style)
entry_deflection_limit = ttk.Entry(app, background=entry_bg_color, font=font_style)
entry_gamma_M = ttk.Entry(app, background=entry_bg_color, font=font_style)
entry_steel_class = ttk.Entry(app, background=entry_bg_color, font=font_style)

entry_force_vertical.grid(row=0, column=1, padx=5, pady=5)
entry_force_horizontal.grid(row=1, column=1, padx=5, pady=5)
entry_length.grid(row=2, column=1, padx=10, pady=5)
entry_depth.grid(row=3, column=1, padx=10, pady=5)
entry_width.grid(row=4, column=1, padx=10, pady=5)
entry_deflection_limit.grid(row=5, column=1, padx=10, pady=5)
entry_gamma_M.grid(row=6, column=1, padx=10, pady=5)
entry_steel_class.grid(row=7, column=1, padx=5, pady=5)

# Option to select file
selected_file = tk.IntVar()
selected_file.set(1)  # Default selection
ttk.Radiobutton(app, text='Version 1 - set with hollow sections', variable=selected_file, value=1).grid(row=len(labels) + 3, column=0, sticky='w', padx=10, pady=5)
ttk.Radiobutton(app, text='Version 2 - set without hollow sections', variable=selected_file, value=2).grid(row=len(labels) + 4, column=0, sticky='w', padx=10, pady=5)

result_text = tk.StringVar()
result_label = ttk.Label(app, textvariable=result_text, background=bg_color, font=font_style)
result_label.grid(row=len(labels), columnspan=10, pady=20, padx=10)

# Function to clear input fields
def clear_entries():
    entry_force_vertical.delete(0, 'end')
    entry_force_horizontal.delete(0, 'end')
    entry_length.delete(0, 'end')
    entry_depth.delete(0, 'end')
    entry_width.delete(0, 'end')
    entry_deflection_limit.delete(0, 'end')
    entry_gamma_M.delete(0, 'end')
    entry_steel_class.delete(0, 'end')
    result_text.set("")  # Clear result label

# Create a button to calculate beam parameters
calculate_button = ttk.Button(app, text="Calculate", command=calculate_beam, style='Green.TButton')
calculate_button.grid(row=len(labels) + 1, columnspan=5, pady=20)

# Create a button to clear input fields
clear_button = ttk.Button(app, text="Clear", command=clear_entries, style='Red.TButton')
clear_button.grid(row=len(labels) + 2, columnspan=2, pady=10)

# Styling for buttons
style = ttk.Style()
style.configure('Green.TButton', background=button_bg_color, foreground=button_fg_color)
style.configure('Red.TButton', background='red3', foreground=button_fg_color)  # Red button for clearing entries

# Run the main event loop
app.mainloop()


# 